In [1]:
import pickle
import os
import torch
import numpy as np
import IPython.display as ipd
from util.evaluate import Evaluator
from factory.AutoVC_Adjust import AutoVC_Adjust

In [2]:
class Config:
    def __init__(self,root):
        self.root = root
        self.num_speaker = 40
        self.batch_size = 2
        self.erroment_uttr_idx = 16
        self.max_uttr_idx = 68
        self.len_crop = 176
        self.device = "cuda:0"
        self.x_fid_style = np.load("model/static/x_style.npy",allow_pickle=True)
        self.judge = torch.load('model/static/metadv_en_openslr.pt').to(self.device)
        self.all_speaker = sorted(next(iter(os.walk(root)))[1][:self.num_speaker])
        self.metadata = pickle.load(open(f'{root}/train.pkl', "rb"))[:self.num_speaker]

In [3]:
config = Config('train_data_vctk_ver2')

In [4]:
org_ad = AutoVC_Adjust(44,256,512,22).to(config.device)
org_ad.load_state_dict(torch.load("model/autovc_adjust_gan_en.pt", map_location=config.device))

<All keys matched successfully>

In [5]:
E = Evaluator(config)

Detect Judge ! generate all Real Data d-vector
Processing --- ID:0 Speaker:p225 ---
Processing --- ID:1 Speaker:p226 ---
Processing --- ID:2 Speaker:p227 ---
Processing --- ID:3 Speaker:p228 ---
Processing --- ID:4 Speaker:p229 ---
Processing --- ID:5 Speaker:p230 ---
Processing --- ID:6 Speaker:p231 ---
Processing --- ID:7 Speaker:p232 ---
Processing --- ID:8 Speaker:p233 ---
Processing --- ID:9 Speaker:p234 ---
Processing --- ID:10 Speaker:p236 ---
Processing --- ID:11 Speaker:p237 ---
Processing --- ID:12 Speaker:p238 ---
Processing --- ID:13 Speaker:p239 ---
Processing --- ID:14 Speaker:p240 ---
Processing --- ID:15 Speaker:p241 ---
Processing --- ID:16 Speaker:p243 ---
Processing --- ID:17 Speaker:p244 ---
Processing --- ID:18 Speaker:p245 ---
Processing --- ID:19 Speaker:p246 ---
Processing --- ID:20 Speaker:p247 ---
Processing --- ID:21 Speaker:p248 ---
Processing --- ID:22 Speaker:p249 ---
Processing --- ID:23 Speaker:p250 ---
Processing --- ID:24 Speaker:p251 ---
Processing --

In [ ]:
all_fid = E.get_fid_result(org_ad,True)

Processing p225
Processing p226


In [ ]:
np.save("fid",all_fid)

In [ ]:
real_cos_result,real_fid_result = E.get_real_data_result()

In [ ]:
real_fid_result

In [ ]:
np.diagonal(real_cos_result).mean(), (np.sum(real_cos_result) - (np.sum(np.diagonal(real_cos_result)))) /(config.num_speaker*(config.num_speaker-1))

In [ ]:
all_cos_result,all_fid = E.generate_result([org_ad],True)

In [ ]:
all_fid[0][0].mean(),all_fid[0][1].mean(),all_fid[0][2].mean(),all_fid[0][3].mean()

In [ ]:
all_rc_result = []
all_o_rc_result = []
all_trans_result = []
all_o_trans_result = []
fid_result = []
for _ in range(1):
    all_cos_result = E.generate_result([org_ad],True)
    rc,o_rc = E.get_cos_rc(all_cos_result[0])
    trans,o_trans = E.get_cos_trans(all_cos_result[0])
    all_rc_result.append(rc)
    all_o_rc_result.append(o_rc)
    all_trans_result.append(trans)
    all_o_trans_result.append(o_trans)

In [ ]:
np.array(all_rc_result).mean(),np.array(all_trans_result).mean()

In [ ]:
np.array(all_o_rc_result).mean(),np.array(all_o_trans_result).mean()

In [ ]:
np.array(all_rc_result).std(),np.array(all_trans_result).std()

In [ ]:
np.array(all_o_rc_result).std(),np.array(all_o_trans_result).std()